# Training Classifiers
### Procedures:
- Add your collected dataset to variable [your_org]_dataset
- Split your dataset by article in to tuples of (article, 'news_org') and append to docs
- Tokenize words, then append them to all_words
- In the Visualization cell, add your news org to the classes list

In [4]:
import nltk
import random 
import numpy
import scipy
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier

#############################################
# ADD YOUR DATASET HERE
cnn_dataset = open('./Datasets/clean_cnn_dataset', 'r', encoding='utf-8').read()
bbc_dataset = open('./Datasets/clean_bbc_dataset', 'r', encoding='utf-8').read()
nyt_dataset = open('./Datasets/clean_nyt_dataset', 'r', encoding='utf-8').read()
fox_dataset = open('./Datasets/clean_fox_dataset', 'r', encoding='utf-8').read()
alj_dataset = open('./Datasets/clean_alj_dataset', 'r', encoding='utf-8').read()

#############################################

#def clean_dataset(dataset):
#    cleanset = []
#    for word in dataset:
#        if word not in stopwords:
#            cleanset.append(word)
#    return cleanset

#cnn_cleanset = clean_dataset(cnn_dataset)
#bbc_cleanset = clean_dataset(bbc_dataset)

docs = []
all_words = []

#################################################
# APPEND TUPLES TO DOC
for r in cnn_dataset.split('\n'):
    docs.append((r, 'cnn'))
    
for r in bbc_dataset.split('\n'):
    docs.append((r, 'bbc'))
    
for r in nyt_dataset.split('\n'):
    docs.append((r, 'nyt'))
    
for r in fox_dataset.split('\n'):
    docs.append((r, 'fox'))
    
for r in alj_dataset.split('\n'):
    docs.append((r, 'alj'))
    


# TOKENIZE    
cnn_words = nltk.word_tokenize(cnn_dataset)
bbc_words = nltk.word_tokenize(bbc_dataset)
nyt_words = nltk.word_tokenize(nyt_dataset)
fox_words = nltk.word_tokenize(fox_dataset)
alj_words = nltk.word_tokenize(alj_dataset)

# APPEND LOWERCASE WORDS TO ALL WORDS
for w in cnn_words:
    all_words.append(w.lower())
    
for w in bbc_words:
    all_words.append(w.lower())

for w in nyt_words:
    all_words.append(w.lower())
    
for w in fox_words:
    all_words.append(w.lower())
    
for w in alj_words:
    all_words.append(w.lower())
    
###################################################    
    
all_words = nltk.FreqDist(all_words)
word_features = list(all_words.keys())

def find_features(doc):
    words = nltk.word_tokenize(doc)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

featuresets = [(find_features(art), org) for (art, org) in docs]
random.shuffle(featuresets)

# CHANGE SIZE OF TRAINING AND TESTING SETS HOW YOU SEE FIT
print('Length of featuresets:', len(featuresets))
train_set, test_set = featuresets[200:], featuresets[:200]

Length of featuresets: 1402


## Naive Bayes Classifier

In [5]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
print("NB Acc:", nltk.classify.accuracy(classifier, test_set) * 100)
classifier.show_most_informative_features(25)

NB Acc: 68.0
Most Informative Features
                      al = True              alj : fox    =    141.5 : 1.0
               messenger = True              cnn : alj    =    114.9 : 1.0
                       ” = True              nyt : bbc    =     96.5 : 1.0
                     mr. = True              nyt : bbc    =     92.8 : 1.0
                       “ = True              nyt : alj    =     92.0 : 1.0
                       , = False             fox : nyt    =     90.1 : 1.0
                 unfolds = True              cnn : nyt    =     85.1 : 1.0
                       . = False             fox : nyt    =     84.0 : 1.0
                       — = True              nyt : bbc    =     84.0 : 1.0
                     cnn = True              cnn : fox    =     83.7 : 1.0
                reserved = True              fox : alj    =     79.5 : 1.0
                      et = True              cnn : fox    =     76.4 : 1.0
                       t = True              nyt : bbc    =  

## Stochastic Gradient Descent Classifier

In [6]:
SGD_classifier = SklearnClassifier(SGDClassifier())
SGD_classifier.train(train_set)
print("SGD Acc:", nltk.classify.accuracy(SGD_classifier, test_set) * 100)


SGD Acc: 92.5


## Visualization

In [7]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools
import numpy as np

#print('Size of test_set', len(test_set))
#for test in test_set:
#    if(test[1] == 'fox'):
#        print(test[0])
        
ref = [org for (art, org) in test_set]
test = [SGD_classifier.classify(art) for (art, org) in test_set]

cm = nltk.ConfusionMatrix(ref, test)
print(cm.pretty_format(sort_by_count=True, show_percents=True, truncate=9))


'''
sklcm = confusion_matrix(ref, test)
print(sklcm)
np.set_printoptions(precision=2)

def plot_cm(cm, classes, title, cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    plt.tight_layout()
    plt.ylabel('True Class')
    plt.xlabel('Predicted Class')
    
plt.figure()
plot_cm(sklcm, classes=['bbc', 'nyt', 'cnn'], title='SGD Classifier')
plt.show()
'''

    |      f      n      a      b      c |
    |      o      y      l      b      n |
    |      x      t      j      c      n |
----+------------------------------------+
fox | <21.0%>  1.5%      .   0.5%   1.0% |
nyt |   1.0% <23.0%>     .      .      . |
alj |   0.5%   0.5% <18.5%>  1.0%   1.0% |
bbc |      .      .      . <20.5%>  0.5% |
cnn |      .      .      .      .  <9.5%>|
----+------------------------------------+
(row = reference; col = test)



"\nsklcm = confusion_matrix(ref, test)\nprint(sklcm)\nnp.set_printoptions(precision=2)\n\ndef plot_cm(cm, classes, title, cmap=plt.cm.Blues):\n    plt.imshow(cm, interpolation='nearest', cmap=cmap)\n    plt.title(title)\n    plt.colorbar()\n    tick_marks = np.arange(len(classes))\n    plt.xticks(tick_marks, classes, rotation=45)\n    plt.yticks(tick_marks, classes)\n    plt.tight_layout()\n    plt.ylabel('True Class')\n    plt.xlabel('Predicted Class')\n    \nplt.figure()\nplot_cm(sklcm, classes=['bbc', 'nyt', 'cnn'], title='SGD Classifier')\nplt.show()\n"

## Save Classifiers

In [8]:
# Save Classifiers
import pickle

pickle.dump(classifier, open('./Classifiers/classifier.pickle', 'wb'))
pickle.dump(SGD_classifier, open('./Classifiers/sgd_classifier.pickle', 'wb'))